<a href="https://colab.research.google.com/github/KessonDalef/Face2face1024_colab_demo/blob/master/face2face_1024px_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# face2face 1024 px

**A Google Colab for the face2face 1024 edited by Karol Majek to work with high resolution images.**

Read his great article on Medium [HERE](https://medium.com/@karol_majek/high-resolution-face2face-with-pix2pix-1024x1024-37b90c1ca7e8)

### Prepare 

- STEP 1: Clone the repo

In [0]:
!git clone git clone https://github.com/karolmajek/face2face-demo.git --recursive

- STEP 2: Mount your Google Drive to access its files

*Reasons: the reason to use Google Drive is that, first of all, you can easily import a video you want to use for the face2face from there, and then you can put your Drive folder as output for the model so if the Runtime disconnects for any reasons (it happened to me many times and I got very frustrated), you have the checkpoint saved in your Drive that you can always restore.*

In [0]:
from google.colab import drive
drive.mount('/content/drive')

- STEP 3: Import the video from Drive

*You can also right click in the menu on the eft and upload in Colab, it would get deleted in the RUntime disconnects*

In [0]:
# My video was in the main folder of Drive
!cp "/content/drive/My Drive/NeuralNetworks/video.mp4" "/content/faceface-demo/video.mp4" 

### Generate Training Data

- STEP 4: Generate the training data

**FROM THE REPO:**
  
  INPUT: 
- **file** *is the name of the video file from which you want to create the data set.*
- **num** *is the number of train data to be created.*
- **landmark-model** *is the facial landmark model that is used to detect the landmarks. A pre-trained facial landmark model is provided* [here](http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2). *Download it and upload by right clicking in the left panel and choose "Upload file"*.

OUTPUT: 

*Two folders* **original** *and* **landmarks** *will be created.*

In [0]:
# Enter in the working directory
%cd face2face-demo

# And generate the data
!python generate_train_data.py --file video.mp4 --num 400 --landmark-model shape_predictor_68_face_landmarks.dat

- STEP 5: This step is not mandatory and you can skip it, we will now make a backup of the output of the previous step in our Google Drive, so in the case we want to recover from checkpoint we don't need to generate the training data, but we will copy them from our Drive

*If you don't want to copy the generated data on your Drive, or you don't need/want to import the data from your Drive, you can skip steps 5 and 6.*

In [0]:
# A new folder named original will be made in our Google Drive
!cp -r "./original" "/content/drive/My Drive"

# And also a new folder called landmarks will be made in our Drive
!cp -r "./landmarks" "/content/drive/My Drive"

### Training

- STEP 6: If you want to import data from your Google Drive generated in the past, let's do it now.

In [0]:
# The original images
!cp -r "/content/drive/My Drive/original" "/content/face2face-demo/"

# And the landmarks
!cp -r "/content/drive/My Drive/landmarks" "/content/face2face-demo/"

- STEP 7: Let's copy our dataset in the pix2pix-tensorflow directory

In [0]:
# Create the photo folder
!mkdir pix2pix/photos

# Copy the data from the working directory to the pix2pix dir
!cp -r ./original pix2pix-tensorflow/photos/
!cp -r ./landmarks pix2pix-tensroflow/photos/

# Now let's remove useless data
!rm -r ./original
!rm -r ./landmarks

- STEP 8: Let's move to the pix2pix dir

In [0]:
%cd pix2pix-tensorflow

- STEP 9: Resize the original images

In [0]:
!python tools/process.py \
  --input_dir photos/original \
  --operation resize \
  --size 1024 \
  --output_dir photos/original_resized

- STEP 10: Resize landmark images

In [0]:
!python tools/process.py \
  --input_dir photos/landmarks \
  --operation resize \
  --size 1024 \
  --output_dir photos/landmarks_resized

- STEP 11: Combine both resized original and landmark images

In [0]:
!python tools/process.py \
  --input_dir photos/landmarks_resized \
  --b_dir photos/original_resized \
  --operation combine \
  --output_dir photos/combined

- STEP 12: Split into train/val set

In [0]:
!python tools/split.py \
  --dir photos/combined

- STEP 13: Train the model on the data

**output_dir** *is the directory where the checkpoints of the model will be saved, and is the folder where you can restore. In this case, if you mounted Google Drive before, you can choose your Drive folder as output. It will be slightly slower, but we can be sure to not to lose the data in the case of disconnection.*

*For more informations, check the original repo of this project:*
https://github.com/karolmajek/face2face-demo

*Uncomment the last line by removing the # if you want to use the checkpoint from Drive.*

In [0]:
!python pix2pix.py \
  --mode train \
  --output_dir "/content/drive/My Drive/face2face-model" \
  --max_epochs 200 \
  --input_dir photos/combined/train \
  --which_direction AtoB #\
#  --checkpoint "/content/drive/My Drive/face2face-model"

- STEP 14: This step is to test the xported model and for the validation. If you are not interested in testing the model, you can skip this step.

*INPUT:*
    
- **output_dir** *is the output folder of our test, an HTML page with some tests on the val folder*
- **input_dir** *is the fodler where we stored the "Val" data for the evaluation, check the splitting step*
- **checkpoint** *is the output_dir where we saved the checkpoint. In this case, is set to that folder in Goolgle Drive, but you can change it with what you are using.*

In [0]:
!python pix2pix.py \
  --mode test \
  --output_dir face2face-test \
  --input_dir photos/combined/val \
  --checkpoint "/content/drive/My Drive/face2face-model"

- STEP 15: In the case you went for the test, you can zip and download the folder to open it on your computer.

In [0]:
# Let's zip the folder
!zip -r ./face2face-test face2face-test.zip

# Import file manager from COlab
from google.colab import files

# And download the zip file, it might take a while to prepare the download
files.download("./face2face-test")

- STEP 16: If we saved our model on Drive, let's copy it in our working directory to speed up the process

*If you didn't save the model on Drive skip this passage.*

In [0]:
!cp -r "/content/drive/My Drive/face2face-model" face2face-model

### Export Model

- STEP 17: First, we need to reduce the trained model so that we can use an image tensor as input

**FROM THE REPO:**

INPUT:
- **model-input** *is the model folder to be imported.*
- **model-output** *is the model (reduced) folder to be exported.*

OUTPUT:
- It returns a reduced model with less weights file size than the original model.

In [0]:
!python ../reduce_model.py --model-input face2face-model --model-output face2face-reduced-model 

- STEP 18: Second, we freeze the reduced model to a single file.

**FROM THE REPO:**

INPUT:
- **model-folder** *is the model folder of the reduced model.*

OUTPUT:
- *It returns a frozen model file* **frozen_model.pb** *in the model folder.*


In [0]:
!python ../freeze_model.py --model-folder face2face-reduced-model

### Run the demo

**This Colab is made only for the training!**

Now that we trained our model, it's time to download it and use it on out machine.

The best way is to follow the steps from the official repository and make it working on our machine. Run the next code to download the frozen-model. If you would like to download the entire model, uncomment the lines in orage by removing the # at the beginning

In [0]:
from google.colab import files
# zip -r face2face-model.zip face2face-model
# files.download("face-face-model.zip")
zip -r face2face-reduced-model.zip face2face-reduced-model
files.download("face2face-reduced-model.zip")

### That's it. Hope you enjoyed!